# 2_1_ClimateTuning
Before we can work on forecasting and tuning with the model, we will have to generate a climate forecasting model. As part of the forecasting challenge, there is a gap between the input data and the output. Essentially, we can only use data up to EW 25 of year $t$ and we need to forecast for EW 41 year $t$ to EW40 year $t+1$. This gap includes both dengue and climate data. We try tuning three different models here: (1) linear regression, (2) random forest, (3) XGBoost, focussing only on the lags hyperparameter.

In [1]:
import pandas as pd
import numpy as np
import torch 
import os
import json

from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar
from darts.metrics import mape, smape, mse, rmse, mae, mql, ql
from darts.dataprocessing.transformers import Scaler
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from sklearn.preprocessing import MinMaxScaler
import optuna
from optuna.samplers import TPESampler
from darts.models import (
    RandomForestModel,
    LinearRegressionModel,
    XGBModel
)
from darts.utils.likelihood_models import GaussianLikelihood, PoissonLikelihood, QuantileRegression
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import Callback, EarlyStopping
import pickle
pd.options.mode.copy_on_write = True
random_seed = 20


## 0. Helper Functions and Globals

In [2]:
clim_var_names = ["temp_min", "temp_med", "temp_max", "precip_min", "precip_med", "precip_max", "pressure_min", "pressure_med", "pressure_max", "rel_humid_min", "rel_humid_med", "rel_humid_max"]

In [3]:
def plot_mv_series(obs_ts, preds_ts, curr_fig_dims, curr_val_cols, width, height, train_ts = None, train_ts_length = None): 
    fig, axs = plt.subplots(curr_fig_dims[0], curr_fig_dims[1], figsize = (width,  height))
    for curr_col, curr_ax in zip(curr_val_cols, axs.flatten()):
        if train_ts is not None:
            if train_ts_length is not None:
                train_ts[curr_col][-train_ts_length:].plot(label = "Training", ax = curr_ax)
            else:
                train_ts[curr_col].plot(label = "Training", ax = curr_ax)
        obs_ts[curr_col].plot(label = "Observed", ax = curr_ax)
        preds_ts[curr_col].plot(label = "Forecasted", ax = curr_ax)
        curr_ax.set_title(curr_col)
        curr_ax.set_xlabel("")


def log_bound(x, lower, upper, plus_1 = True, eps = 1e-6):
    """
    Transforms some value x such that when we forecast it, it will always be between lower and upper
    """
    if plus_1: 
        x += 1
        lower += 1
        upper += 1
        
    y = np.log1p((x - lower)/(upper - x + eps))
    return y

def inv_log_bounds(y, lower, upper, plus_1 = True, eps = 1e-6):
    """
    Inverse of the log_bounds transformation above
    """
    exp_y = np.exp(y)
    numerator = (exp_y - 1) * (upper + eps) + lower
    denominator = exp_y
    return numerator / denominator

def generate_preds_obs_plot(train_list, obs_list, preds_list, uf_list, uf_sub_list, clim_var_names = clim_var_names, label_list = None, width = 20, height = 42, show_train = True):
    to_vis_train = [curr_ts for curr_ts, curr_uf in zip(train_list, uf_list) if curr_uf in uf_sub_list]
    to_vis_preds_list = [curr_ts for curr_ts, curr_uf in zip(preds_list, uf_list) if curr_uf in uf_sub_list]
    to_vis_obs_list = [curr_ts for curr_ts, curr_uf in zip(obs_list, uf_list) if curr_uf in uf_sub_list]
    fig, axs = plt.subplots(len(clim_var_names), len(uf_sub_list), figsize = (width,  height))
    if label_list is None:
        label_list = uf_sub_list
    else:
        label_list = [curr_label for curr_label, curr_uf in zip(label_list, uf_list) if curr_uf in uf_sub_list]
    for ind1, (curr_train, curr_obs, curr_preds, curr_label) in enumerate(zip(to_vis_train, to_vis_obs_list, to_vis_preds_list, label_list)):
        for ind2, curr_var in enumerate(clim_var_names):
            if show_train:
                curr_train[curr_var].plot(ax = axs[ind2][ind1], label = "Training")
            curr_obs[curr_var].plot(ax = axs[ind2][ind1], label = "Observed")
            curr_preds[curr_var].plot(ax = axs[ind2][ind1], label = "Forecasted")
            axs[ind2][ind1].set_xlabel("")
            axs[ind2][ind1].set_ylabel(curr_var)
            axs[ind2][ind1].set_title(curr_label)

def norm_metric(curr_obs, curr_preds, curr_normer, summ_func = None, metric = mae):
    """
    Computes a normalised version of some Darts metric given an observed, predicted, and normalising TimeSeries.
    Note that this function assumes that all inputs are on their natural scales (should invert any log transforms or min-max scaling)

    Parameters:
    curr_obs (TimeSeries): Darts TimeSeries of observed values to compute metrics against
    curr_preds (TimeSeries): Darts TimeSeries forecasted values to commpute metrics with
    curr_normer (TimeSeries): Darts TimeSeries that serves as the basis of min-max values to normalise metrics (usually the training set)
    summ_func (function): Function to summarise the normalised per component metrics
    metric (function): Darts metric function (see: https://unit8co.github.io/darts/generated_api/darts.metrics.metrics.html)

    Returns:
    Value of the computed metric summarised (if summ_func is not None) or an array containing the metric per component (if summ_func is None)
    """
    curr_preds = curr_preds.slice_intersect(curr_obs) #Get the part of the forecast that intersects with the observation
    assert len(curr_preds) == len(curr_obs) #Check that the forecasts and observations match in length
    
    bounds_list = [] 
    for comp in curr_normer.components: #Use the normer series to find the min and max per component
        values = curr_normer[comp].values()
        comp_min = values.min()
        comp_max = values.max()
        bounds_list.append((comp_min, comp_max))
    
    met_per_comp = metric(curr_obs, curr_preds, component_reduction = None) #Get the metric value per component
    norm_met_vals = []
    for met, min_max in zip(met_per_comp, bounds_list):
        curr_min = min_max[0]
        curr_max = min_max[1]
        norm_met_vals.append(met / (curr_max - curr_min)) #Normalise the component error based on the min and max for each component
    norm_met_vals = np.array(norm_met_vals)

    #Whether to summarise the metric values or to just return the whole array
    if summ_func is None:
        return norm_met_vals
    else:
        return summ_func(norm_met_vals)

def norm_metric_list(obs_list, preds_list, normer_list, comp_summ_func = None, series_summ_func = None, metric = mae):
    """
    Computes a normalised metric using a list of observed, predicted, and normalising series. Function assumes that all series are on
    their natural scales. 

    Parameters:
    obs_list (list): List of Darts TimeSeries containing observed values to compute metrics against
    preds_list (list): List of Darts TimeSeries containing forecasted values to commpute metrics with
    normer_list (list): List of Darts TimeSeries that serve as the basis of min-max values to normalise metrics (usually the training set)
    comp_summ_func (function): Function to summarise the normalised per component metrics
    series_summ_func (function): Function to summarise metrics across different series
    metric (function): Darts metric function (see: https://unit8co.github.io/darts/generated_api/darts.metrics.metrics.html)
    """
    result = []
    for curr_obs, curr_preds, curr_normer in zip(obs_list, preds_list, normer_list):
        met_val = norm_metric(curr_obs, curr_preds, curr_normer, summ_func = comp_summ_func, metric = metric)
        result.append(met_val)
    result = np.array(result)

    if series_summ_func is None:
        return result
    else:
        return series_summ_func(result)

#def generate_component_metrics_table(obs_list, preds_list, label_list, label_name, component_names, normer_list = None, metrics = [mae], metric_names = ["MAE"], disp_mode = False):
def generate_component_metric_table(obs_list, preds_list, label_list, label_name, component_names, curr_metric, normer_list = None, disp_mode = False):
    """
    Function generates a table meant to display the error per state per component
    """
    table_builder = []
    if normer_list is not None: 
        metric_res = norm_metric_list(obs_list, preds_list, normer_list, metric = curr_metric)
    else:
        metric_res = curr_metric(obs_list, preds_list, component_reduction = None)
    
    for curr_label, curr_errors in zip(label_list, metric_res):
        curr_dict = {label_name: curr_label}
        for curr_comp, curr_comp_error in zip(component_names, curr_errors):
            curr_dict[curr_comp] = curr_comp_error
        table_builder.append(curr_dict)
        
    curr_table = pd.DataFrame(table_builder)
    
    if disp_mode:
        disp_table = curr_table.copy()
        summ_row = {label_name: "Summary"}
        for curr_comp in component_names:
            central = np.mean(curr_table[curr_comp])
            lower = np.min(curr_table[curr_comp])
            upper = np.max(curr_table[curr_comp])
            summ_row[curr_comp] = "%.3f (%.3f to %.3f)" % (central, lower, upper)
            disp_table[curr_comp] = disp_table.apply(lambda row: "%.3f" % row[curr_comp], axis = 1)
        disp_table = pd.concat([disp_table, pd.DataFrame([summ_row])])
        return disp_table
    
    return curr_table

def generate_series_metrics_table(obs_list, preds_list, label_list, label_name, metrics = [mae, rmse], metric_names = ["MAE", "RMSE"],  normer_list = None, disp_mode = False):
    """
    Function displays multiple error metrics (each the mean across components) per state
    """
    if normer_list is not None:
        metric_names = ["Norm_" + curr_name for curr_name in metric_names]
    curr_table = pd.DataFrame([{label_name: curr_label} for curr_label in label_list])
    for curr_metric, curr_metric_name in zip(metrics, metric_names):
        if normer_list is not None: 
            curr_res = norm_metric_list(obs_list, preds_list, normer_list, comp_summ_func = np.mean, metric = curr_metric)
        else:
            curr_res = metric(obs_list, preds_list)
        curr_table[curr_metric_name] = curr_res
    
    if disp_mode:
        disp_table = curr_table.copy()
        summ_row = {label_name: "Summary"}
        for curr_metric_name in metric_names:
            central = np.mean(curr_table[curr_metric_name])
            lower = np.min(curr_table[curr_metric_name])
            upper = np.max(curr_table[curr_metric_name])
            summ_row[curr_metric_name] = "%.3f (%.3f to %.3f)" % (central, lower, upper)
            disp_table[curr_metric_name] = disp_table.apply(lambda row: "%.3f" % row[curr_metric_name], axis = 1)
            
        disp_table = pd.concat([disp_table, pd.DataFrame([summ_row])])
        return disp_table
        
    return curr_table

def dict_as_json(curr_json, path):
    with open(path, "w") as f:
        json.dump(curr_json, f, indent = 4)

def save_pickle(to_save, save_path):
    with open(save_path, "wb") as file:
        pickle.dump(to_save, file)

def load_pickle(load_path):
    to_ret = None 
    with open(load_path, "rb") as file:
        to_ret = pickle.load(file)
    return(to_ret)


## 1. Reading Input
First, we read in the input files: (1) a weekly calendar - which also includes flags for inclusion in the different data sets, (2) SST Indices, (3) Climate data.

In [4]:
global_scale = False
retrain_per_fold = True
approach = "forecast_gap"

tuning_name = "Notebook_24_07_2025"
base_dir = os.getcwd()
model_input_dir = os.path.join(base_dir, "ModelInput")
tuning_dir = os.path.join(base_dir, "Climate Tuning", tuning_name)

os.makedirs(tuning_dir, exist_ok = True)

tuning_details = {"clim_var_names": clim_var_names, "global_scale": global_scale, "retrain_per_fold": retrain_per_fold, "approach": approach}
dict_as_json(tuning_details, os.path.join(tuning_dir, "tuning_details.json"))

In [5]:
weekly_cal = pd.read_csv(os.path.join(model_input_dir, "Calendar.csv"))
sst_df = pd.read_csv(os.path.join(model_input_dir, "SSTIndices.csv"))
climate_df = pd.read_csv(os.path.join(model_input_dir, "TimeVaryingCovs.csv"))

#Remove uf = "ES" since there is no dengue data for Espirito Santo
climate_df = climate_df[climate_df["uf"] != "ES"]

In [6]:
#Set info where validation sets in the main dengue forecasting part occur only during the dengue season W41-40 year t to t+1. 
set_info = weekly_cal[["epiweek", "Year", "Week", "WeekStart", "WeekMid", "WeekEnd"]]
set_info = set_info.iloc[1:]

#Define the data sets using a column of booleans in the calendar. 
#all epiweek numbers are in YYYYWW integer format
set_info["clim_train1"] = set_info["epiweek"] <= 201510
set_info["clim_val1"] = (set_info["epiweek"] >= 201526) & (set_info["epiweek"] <= 201625)
set_info["clim_train2"] = set_info["epiweek"] <= 201610
set_info["clim_val2"] = (set_info["epiweek"] >= 201626) & (set_info["epiweek"] <= 201725)
set_info["clim_train3"] = set_info["epiweek"] <= 201710
set_info["clim_val3"] = (set_info["epiweek"] >= 201726) & (set_info["epiweek"] <= 201825)
set_info["clim_test"] = (set_info["epiweek"] >= 201841) & (set_info["epiweek"] <= 201940)
set_info["clim_test_input"] = set_info["epiweek"] <= 201825
set_info

,epiweek,Year,Week,WeekStart,WeekMid,WeekEnd,clim_train1,clim_val1,clim_train2,clim_val2,clim_train3,clim_val3,clim_test,clim_test_input
1,201001,2010,1,2010-01-03,2010-01-07,2010-01-09,True,False,True,False,True,False,False,True
2,201002,2010,2,2010-01-10,2010-01-14,2010-01-16,True,False,True,False,True,False,False,True
3,201003,2010,3,2010-01-17,2010-01-21,2010-01-23,True,False,True,False,True,False,False,True
4,201004,2010,4,2010-01-24,2010-01-28,2010-01-30,True,False,True,False,True,False,False,True
5,201005,2010,5,2010-01-31,2010-02-04,2010-02-06,True,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,202523,2025,23,2025-06-01,2025-06-05,2025-06-07,False,False,False,False,False,False,False,False
806,202524,2025,24,2025-06-08,2025-06-12,2025-06-14,False,False,False,False,False,False,False,False
807,202525,2025,25,2025-06-15,2025-06-19,2025-06-21,False,False,False,False,False,False,False,False
808,202526,2025,26,2025-06-22,2025-06-26,2025-06-28,False,False,False,False,False,False,False,False


##  2. Creating TimeSeries from Climate Input

In [7]:
def create_ts(data_df, val_cols, set_info = None, set_column = None, time_col = "WeekStart", time_format = "%Y-%m-%d"):
    """
    Function creates a TimeSeries object from a DataFrame using only rows where set_column is true

    Parameters:
    data_df (DataFrame): DataFrame containing TimeSeries data (assumed in chronological order)
    val_cols (list or string): Name(s) of the column(s) in data_df containing the values to make a TimeSeries (will be multivariate if a list with more than 1 element)
    set_info (DataFrame): DataFrame to merge into data_df (assumed with Year, Week, and epiweek columns) containg set information
    set_column (string): Name of the column containing booleans to be used for inclusion/exclusion in the output TimeSeries
    time_col (string): Name of the column to be used as the time-axis of the output series
    time_format (string): Format of the time column

    Returns:
    curr_ts: TimeSeries filtered based on set_column with values based on val_cols
    """
    data_df[time_col] = pd.to_datetime(data_df[time_col], format = time_format)
    data_df = data_df.set_index(time_col)

    if set_info is not None:
        #We assume the set_info DataFrame has Year, Week, and epiweek columns we can join on 
        data_df = pd.merge(data_df, set_info, how = "left", on = ["Year", "Week", "epiweek"]) 
    if set_column is not None:
        data_df = data_df[data_df[set_column]]
    
    curr_ts = TimeSeries.from_times_and_values(times = data_df.index, values = data_df[val_cols], columns = val_cols)
    return curr_ts


def create_ts_list(data_df, val_cols, group_col, group_vals, set_info = None, set_column = None, time_col = "WeekStart", time_format = "%Y-%m-%d"):
    """
    Function creates a list TimeSeries objects from a DataFrame. Each list element is a TimeSeries for a particular unit from group_vals, based on the group_col

    Parameters:
    data_df (DataFrame): DataFrame containing TimeSeries data (assumed in chronological order)
    val_cols (list or string): Name(s) of the column(s) in data_df containing the values to make a TimeSeries (will be multivariate if a list with more than 1 element)
    group_col (string): Name of the column to match with the units from group_vals
    group_vals (list): List of values in the group_col to make a TimeSeries for. The list returned by this function will have a TimeSeries for each element in group_vals.
    set_info (DataFrame): DataFrame to merge into data_df (assumed with Year, Week, and epiweek columns) containg set information
    set_column (string): Name of the column containing booleans to be used for inclusion/exclusion in the output TimeSeries
    time_col (string): Name of the column to be used as the time-axis of the output series
    time_format (string): Format of the time column

    Returns:
    to_ret: list of TimeSeries, one for each element in group_vals
    """
    to_ret = []
    if set_info is not None:
        data_df = pd.merge(data_df, set_info, how = "left", on = ["Year", "Week", "epiweek"])
        
    for curr_unit in group_vals:
        curr_unit_df = data_df[data_df[group_col] == curr_unit]
        curr_unit_ts = create_ts(curr_unit_df, val_cols, set_column = set_column, time_col = time_col, time_format = time_format) #No need to pass set_info here since we have already merged in this function
        to_ret.append(curr_unit_ts)
    return to_ret

In [8]:
uf_list = climate_df["uf"].unique()
uf_mapper = climate_df[["uf", "uf_name"]].copy().drop_duplicates()
uf_dict = dict(zip(uf_mapper["uf"], uf_mapper["uf_name"]))
uf_name_list = [uf_dict[curr_uf] for curr_uf in uf_list]

### 2.1 Pre-transforms
Before we create TimeSeries for forecasting, we have to transform some values that have restricted forecast ranges. These are mainly precipitation values (which cannot be below 0) and rainy days (which are restricted from 0 to 7). 

In [9]:
log_trans_cols = ["precip_min", "precip_med", "precip_max"] #Logp1 transform the precipitation values
#logit_trans_cols = ["rainy_days"] #Limit rainy days between 0-7 using our custom logit function
proc_climate_df = climate_df.copy()

#Compute pre-processed version of the climate_df, log-transforming the precipitation variables, which have to always be positive
for curr_col in log_trans_cols:
    proc_climate_df[curr_col] = np.log1p(climate_df[curr_col])

### 2.2 Create TimeSeries
Create the TimeSeries, we have 6 primary ones here, climate train1, val1, train2, val2, train3, and val3 (referred to as Clim 1a,b,c and ClimVal 1a,b,c in documentation). Once we have the 6 series, we then prepare them for modelling by scaling.

In [10]:
def plot_ts_list(plot_list, label_list, plot_comps = None, dims = (7,4), width = 18, height = 24, low_quantile = 0.025, high_quantile = 0.975, fig = None, axs = None, prefixes = None):
    fig, axs = plt.subplots(dims[0], dims[1], figsize = (width,  height))

    if prefixes is None:
        prefixes = ["" for temp in plot_list]

    for ts_list, curr_prefix in zip(plot_list, prefixes):
        for curr_ts, curr_label, curr_ax in zip(ts_list, label_list, axs.flatten()):
            if plot_comps is not None:
                to_plot = curr_ts[plot_comps]
            else:
                to_plot = curr_ts
        
            to_plot.plot(ax = curr_ax, label = curr_prefix, low_quantile = low_quantile, high_quantile = high_quantilte)
            curr_ax.set_title(curr_label)
            curr_ax.set_xlabel("")

#Revert pre-processing
def invert_pre_process_ts(curr_ts, clim_var_names = clim_var_names, log_trans_cols = log_trans_cols, logit_trans_cols = []):
    curr_builder = []
    for curr_var in clim_var_names:
        temp = curr_ts[curr_var] 
        if curr_var in log_trans_cols:
            temp = temp.map(np.expm1)
        elif curr_var in logit_trans_cols:
            temp = temp.map(lambda y: inv_log_bounds(y, 0, 7))
        curr_builder.append(temp)
    
    result_ts = concatenate(curr_builder, axis = 1)
    return result_ts

def invert_pre_process_ts_list(curr_ts_list, clim_var_names = clim_var_names, log_trans_cols = log_trans_cols, logit_trans_cols = []):
    to_ret = []
    for curr_ts in curr_ts_list:
        result = invert_pre_process_ts(curr_ts, clim_var_names = clim_var_names, log_trans_cols = log_trans_cols, logit_trans_cols = logit_trans_cols)
        to_ret.append(result)
    return to_ret

In [11]:
group_col = "uf"

#Training set without the pre-processing, mainly for normalising error metrics
train1 = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_train1")
train2 = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_train2")
train3 = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_train3")

#Training sets with pre-processing for actual model training
train1_log = create_ts_list(proc_climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_train1")
train2_log = create_ts_list(proc_climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_train2")
train3_log = create_ts_list(proc_climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_train3")

#Note that since the validation and test sets are only used for evaluation / error calculation, we use the non pre-processed version (no log-transforms) 
val1 = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_val1")
val2 = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_val2")
val3 = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_val3")

test_input_log = create_ts_list(proc_climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_test_input") # Input sequence for the final model to evaluate on test set
test = create_ts_list(climate_df, clim_var_names, group_col, uf_list, set_info = set_info, set_column = "clim_test") #Test set for evaluation

In [12]:
#Scaling values
clim_scaler1 = Scaler(global_fit = global_scale)
clim_scaler1.fit(train1_log)

#Only scale training sets 
train1_log_s = clim_scaler1.transform(train1_log)
train2_log_s = clim_scaler1.transform(train2_log)
train3_log_s = clim_scaler1.transform(train3_log)

## 3. Model Tuning

In [13]:
def write_study(curr_study, add_path):
    curr_write_dir = os.path.join(tuning_dir, add_path)
    os.makedirs(curr_write_dir, exist_ok = True)
    curr_trial_df = curr_study.trials_dataframe()
    curr_best_params = curr_study.best_params

    dict_as_json(curr_best_params, os.path.join(curr_write_dir, "best_params.json"))
    curr_trial_df.to_csv(os.path.join(curr_write_dir, "trial_df.csv"), index = False)
    save_pickle(curr_study, os.path.join(curr_write_dir, "study_obj.pkl"))

In [14]:
gap_length = 15
#We use 53 as the base forecast length because some years have 53 weeks
if approach == "forecast_gap": #"Forecast the gap" approach
    output_chunk_length = 1 #We assume 1-step ahead for the forecast the gap approach
    output_chunk_shift = 0
    forecast_length = 53 + gap_length
else: #"One shot forecast" approach
    output_chunk_length = 53 #If we are doing a 1-shot forecast, we set output_chunk_length to 53 since some years have 53 weeks.
    output_chunk_shift = gap_length #We set an output_chunk_shift equal to the gap size
    forecast_length = output_chunk_length

In [15]:
def climate_objective(trial, model_mode, retrain_per_fold = retrain_per_fold):
    #Tune the lags hyperparameter
    lags = trial.suggest_int("lags", 4, 208)
    encoders = {"datetime_attribute": {"future": ["month", "year"]}, 
                    "transformer": Scaler()}
    #Create the model, either a LinearRegression, RandomForest, or XGBoost
    if model_mode == "LinearRegression":
        curr_model1 = LinearRegressionModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift)
        curr_model2 = LinearRegressionModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift)
        curr_model3 = LinearRegressionModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift)
    elif model_mode == "RandomForest":
        curr_model1 = RandomForestModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift, random_state = random_seed)
        curr_model2 = RandomForestModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift, random_state = random_seed)
        curr_model3 = RandomForestModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift, random_state = random_seed)
    else:
        curr_model1 = XGBModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift, random_state = random_seed)
        curr_model2 = XGBModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift, random_state = random_seed)
        curr_model3 = XGBModel(lags = lags, lags_future_covariates = (lags, output_chunk_length), add_encoders = encoders, output_chunk_length = output_chunk_length, output_chunk_shift = output_chunk_shift, random_state = random_seed)
    
    #Fit the model using the training set
    curr_model1.fit(series = train1_log_s)

    if retrain_per_fold: #If we retrain per fold, we have to fit curr_model2 and 3, then predict with all three models
        curr_model2.fit(series = train2_log_s)
        curr_model3.fit(series = train3_log_s)
        
        preds1_log_s = curr_model1.predict(series = train1_log_s, n = forecast_length)
        preds2_log_s = curr_model2.predict(series = train2_log_s, n = forecast_length)
        preds3_log_s = curr_model3.predict(series = train3_log_s, n = forecast_length)
    else: #If we do not retrain per fold, all predictions done only with curr_model1
        preds1_log_s = curr_model1.predict(series = train1_log_s, n = forecast_length)
        preds2_log_s = curr_model1.predict(series = train2_log_s, n = forecast_length)
        preds3_log_s = curr_model1.predict(series = train3_log_s, n = forecast_length)

    #Invert the scaling
    preds1_log = clim_scaler1.inverse_transform(preds1_log_s) 
    preds2_log = clim_scaler1.inverse_transform(preds2_log_s)
    preds3_log = clim_scaler1.inverse_transform(preds3_log_s)

    #Invert the log transforms
    preds1 = invert_pre_process_ts_list(preds1_log)
    preds2 = invert_pre_process_ts_list(preds2_log)
    preds3 = invert_pre_process_ts_list(preds3_log)
    
    #Compute the MSE 
    loss1 = norm_metric_list(val1, preds1, train1, comp_summ_func = np.mean, series_summ_func = np.mean, metric = mae)
    loss2 = norm_metric_list(val2, preds2, train1, comp_summ_func = np.mean, series_summ_func = np.mean, metric = mae)
    loss3 = norm_metric_list(val3, preds3, train1, comp_summ_func = np.mean, series_summ_func = np.mean, metric = mae)
    losses = np.array([loss1, loss2, loss3])
    val_loss = np.mean(losses)
    return(val_loss)
    
def print_callback(study, trial):
    print(f"Current Trial: {trial._trial_id}")
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [16]:
sampler1 = TPESampler(seed = random_seed)
study1 = optuna.create_study(direction = "minimize", sampler = sampler1)

study1.optimize(lambda trial: climate_objective(trial, "LinearRegression", retrain_per_fold = retrain_per_fold), callbacks = [print_callback], n_trials = 64)

[I 2025-07-24 22:50:09,275] A new study created in memory with name: no-name-6ae2aeb0-cfd8-41e4-a154-45caeec5503d
[I 2025-07-24 22:50:16,906] Trial 0 finished with value: 213977852707.9635 and parameters: {'lags': 124}. Best is trial 0 with value: 213977852707.9635.


Current Trial: 0
Current value: 213977852707.9635, Current params: {'lags': 124}
Best value: 213977852707.9635, Best params: {'lags': 124}


[I 2025-07-24 22:50:40,234] Trial 1 finished with value: 77322425779.55952 and parameters: {'lags': 188}. Best is trial 1 with value: 77322425779.55952.


Current Trial: 1
Current value: 77322425779.55952, Current params: {'lags': 188}
Best value: 77322425779.55952, Best params: {'lags': 188}


[I 2025-07-24 22:51:03,335] Trial 2 finished with value: 3.925945131754361e+25 and parameters: {'lags': 186}. Best is trial 1 with value: 77322425779.55952.


...

Current Trial: 61
Current value: 0.12265606821145159, Current params: {'lags': 45}
Best value: 0.12129538393256689, Best params: {'lags': 44}


[I 2025-07-24 22:55:33,384] Trial 62 finished with value: 0.12777220630870206 and parameters: {'lags': 54}. Best is trial 10 with value: 0.12129538393256689.


Current Trial: 62
Current value: 0.12777220630870206, Current params: {'lags': 54}
Best value: 0.12129538393256689, Best params: {'lags': 44}


[I 2025-07-24 22:55:35,243] Trial 63 finished with value: 0.13405620443958524 and parameters: {'lags': 21}. Best is trial 10 with value: 0.12129538393256689.


Current Trial: 63
Current value: 0.13405620443958524, Current params: {'lags': 21}
Best value: 0.12129538393256689, Best params: {'lags': 44}


In [17]:
write_study(study1, "LinearRegression")

In [18]:
sampler2 = TPESampler(seed = random_seed)
study2 = optuna.create_study(direction = "minimize", sampler = sampler2)

study2.optimize(lambda trial: climate_objective(trial, "RandomForest", retrain_per_fold = retrain_per_fold), callbacks = [print_callback], n_trials = 64)

[I 2025-07-24 22:55:35,274] A new study created in memory with name: no-name-4c63a34a-ce75-4e59-9ff0-9ad191af9865
[I 2025-07-24 23:33:14,018] Trial 0 finished with value: 0.11530250811253401 and parameters: {'lags': 124}. Best is trial 0 with value: 0.11530250811253401.


Current Trial: 0
Current value: 0.11530250811253401, Current params: {'lags': 124}
Best value: 0.11530250811253401, Best params: {'lags': 124}


[I 2025-07-25 00:12:05,100] Trial 1 finished with value: 0.11218770671079559 and parameters: {'lags': 188}. Best is trial 1 with value: 0.11218770671079559.


Current Trial: 1
Current value: 0.11218770671079559, Current params: {'lags': 188}
Best value: 0.11218770671079559, Best params: {'lags': 188}


[I 2025-07-25 00:50:38,078] Trial 2 finished with value: 0.11184102692473523 and parameters: {'lags': 186}. Best is trial 2 with value: 0.11184102692473523.


...

Current Trial: 61
Current value: 0.11234606872497277, Current params: {'lags': 189}
Best value: 0.11117214070573293, Best params: {'lags': 191}


[I 2025-07-26 09:38:39,066] Trial 62 finished with value: 0.11302115896945715 and parameters: {'lags': 179}. Best is trial 54 with value: 0.11117214070573293.


Current Trial: 62
Current value: 0.11302115896945715, Current params: {'lags': 179}
Best value: 0.11117214070573293, Best params: {'lags': 191}


[I 2025-07-26 10:08:59,009] Trial 63 finished with value: 0.11213827603738231 and parameters: {'lags': 194}. Best is trial 54 with value: 0.11117214070573293.


Current Trial: 63
Current value: 0.11213827603738231, Current params: {'lags': 194}
Best value: 0.11117214070573293, Best params: {'lags': 191}


In [19]:
write_study(study2, "RandomForest")

In [20]:
sampler3 = TPESampler(seed = random_seed)
study3 = optuna.create_study(direction = "minimize", sampler = sampler3)

study3.optimize(lambda trial: climate_objective(trial, "XGBoost", retrain_per_fold = retrain_per_fold), callbacks = [print_callback], n_trials = 64)

[I 2025-07-26 10:08:59,047] A new study created in memory with name: no-name-6679a2fb-12c8-46ba-883c-635ccd91cdf6
[I 2025-07-26 10:15:11,182] Trial 0 finished with value: 0.11935670219708926 and parameters: {'lags': 124}. Best is trial 0 with value: 0.11935670219708926.


Current Trial: 0
Current value: 0.11935670219708926, Current params: {'lags': 124}
Best value: 0.11935670219708926, Best params: {'lags': 124}


[I 2025-07-26 10:23:31,362] Trial 1 finished with value: 0.11629584468518077 and parameters: {'lags': 188}. Best is trial 1 with value: 0.11629584468518077.


Current Trial: 1
Current value: 0.11629584468518077, Current params: {'lags': 188}
Best value: 0.11629584468518077, Best params: {'lags': 188}


[I 2025-07-26 10:31:44,129] Trial 2 finished with value: 0.11590201345706586 and parameters: {'lags': 186}. Best is trial 2 with value: 0.11590201345706586.


...

Current Trial: 61
Current value: 0.114129009713914, Current params: {'lags': 203}
Best value: 0.114129009713914, Best params: {'lags': 203}


[I 2025-07-26 18:56:42,495] Trial 62 finished with value: 0.114129009713914 and parameters: {'lags': 203}. Best is trial 61 with value: 0.114129009713914.


Current Trial: 62
Current value: 0.114129009713914, Current params: {'lags': 203}
Best value: 0.114129009713914, Best params: {'lags': 203}


[I 2025-07-26 19:06:45,753] Trial 63 finished with value: 0.11557369469901091 and parameters: {'lags': 201}. Best is trial 61 with value: 0.114129009713914.


Current Trial: 63
Current value: 0.11557369469901091, Current params: {'lags': 201}
Best value: 0.114129009713914, Best params: {'lags': 203}


In [21]:
write_study(study3, "XGBoost")